In [ ]:
import numpy as np
import os
import pandas
import json
from pathlib import Path
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
import itertools
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# data_dir = "/home/karl/benchmark-project/learn-physics-spring/"
# data_dir = "/home/karl/benchmark-project/learn-integration-spring/"
data_dir = "/home/karl/benchmark-project/wave-architecture-test/"

In [ ]:
aggregate_data = pandas.read_pickle(os.path.join(data_dir, "aggregate_data.pkl"))
ground_truth_data = pandas.read_pickle(os.path.join(data_dir, "ground_truth_data.pkl"))
inferred_data = pandas.read_pickle(os.path.join(data_dir, "inferred_data.pkl"))
aggregate_data.columns
#inferred_data.columns

In [ ]:
pandas.set_option('display.max_rows', 216)
pandas\
.merge(aggregate_data, inferred_data, on="experiment_name")\
.groupby(["method_name", "system_name", "num_train_trajectories", "integrator_timestep_size"])\
.agg({"relerr_l2" : "mean"})

In [ ]:
def make_system_plots(system_name, methods=None, integrator=None, out_of_distribution=None):
    fig, ax = plt.subplots(3, 2)
    fig.set_figwidth(20)
    fig.set_figheight(30)
    
    if not methods:
        methods = ["hnn", "mlp", "knn-regressor", "knn-predictor", "integrator_baseline"]
    
    query_string = "system_name == '{}'".format(system_name)
    if integrator:
        query_string += " and integrator_name == '{}'".format(integrator)
    if out_of_distribution is not None:
        query_string += " and {} experiment_name.str.contains('outdist')".format("" if out_of_distribution else "not")

    if "mlp" in methods or "hnn" in methods:
        df = pandas\
        .merge(aggregate_data, inferred_data, on="experiment_name")\
        .query(query_string)\
        .groupby(["method_name", "integrator_name", "num_train_trajectories", "network_depth", "network_hidden_dim", "timestep_number"])\
        .agg({"relerr_l2" : "mean"}).add_prefix("mean_").reset_index()

    if "mlp" in methods:
        df.loc[df["method_name"] == "mlp"]\
        .pivot(index="timestep_number", columns=["method_name", "integrator_name", "network_depth", "network_hidden_dim", "num_train_trajectories"], values="mean_relerr_l2")\
        .plot(ax=ax[0,0], logy=True)

    if "hnn" in methods:
        df.loc[df["method_name"] == "hnn"]\
        .pivot(index="timestep_number", columns=["method_name", "integrator_name", "network_depth", "network_hidden_dim", "num_train_trajectories"], values="mean_relerr_l2")\
        .plot(ax=ax[0,1], logy=True)

    if "knn-regressor" in methods or "knn-predictor" in methods:
        df = pandas\
        .merge(aggregate_data, inferred_data, on="experiment_name")\
        .query("system_name == '{}'".format(system_name))\
        .groupby(["method_name", "integrator_name", "num_train_trajectories", "timestep_number"])\
        .agg({"relerr_l2" : "mean"}).add_prefix("mean_").reset_index()

    if "knn-regressor" in methods:
        df.loc[df["method_name"] == "knn-regressor"]\
        .pivot(index="timestep_number", columns=["method_name", "integrator_name", "num_train_trajectories"], values="mean_relerr_l2")\
        .plot(ax=ax[1,0], logy=True)

    if "knn-predictor" in methods:
        df.loc[df["method_name"] == "knn-predictor"]\
        .pivot(index="timestep_number", columns=["method_name", "num_train_trajectories"], values="mean_relerr_l2")\
        .plot(ax=ax[1,1], logy=True)

    if "integrator-baseline" in methods:
        df = pandas\
        .merge(aggregate_data, inferred_data, on="experiment_name")\
        .query("system_name == '{}'".format(system_name))\
        .groupby(["method_name", "integrator_name", "timestep_number"])\
        .agg({"relerr_l2" : "mean"}).add_prefix("mean_").reset_index()

        df.loc[df["method_name"] == "integrator-baseline"]\
        .pivot(index="timestep_number", columns=["method_name", "integrator_name"], values="mean_relerr_l2")\
        .plot(ax=ax[2,0], logy=True)

    def style_update(ax):
        max_val = df["mean_relerr_l2"].max()
        ax.set(ylim=(1e-6, 1e1))
        ax.grid()
        ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
        ax.legend(loc="upper left")
        ax.set_xlabel("Timestep number")
        ax.set_ylabel("Relative Error (mean over trajectories)")


    [style_update(ax) for ax in itertools.chain(*ax)]
    [ax_.set_title(title) for ax_, title in zip(itertools.chain(*ax), itertools.chain(["MLP", "HNN"], ["KNN-REGRESSOR", "KNN-PREDICTOR"], ["INTEGRATOR-BASELINE"]))]
    ax[2,1].set_visible(False)
    fig.tight_layout()
    # plt.show()
    # plt.savefig("error_timestep.png")

In [ ]:
make_system_plots("wave", methods=["hnn", "integrator-baseline"], integrator=None, out_of_distribution=None)

In [ ]:
pandas\
.merge(aggregate_data, inferred_data, on="experiment_name")\
.query("system_name == '{}'".format("spring"))\
.groupby(["method_name", "integrator_name"])\
.agg({"inference_time" : "mean"}).add_prefix("mean_").reset_index()



In [ ]:
def plot_system_performance(system_name):
    fig, ax = plt.subplots(1)
    fig.suptitle(system_name)
    fig.set_figwidth(16)
    fig.set_figheight(12)

    df = pandas\
    .merge(aggregate_data, inferred_data, on="experiment_name")\
    .query("system_name == '{}'".format("spring"))\
    .groupby(["method_name", "integrator_name"])\
    .agg({"inference_time" : "mean"}).add_prefix("mean_").reset_index()

    
    df\
    .pivot(index="method_name", columns=["integrator_name"], values="mean_inference_time")\
    .plot(ax=ax, kind="bar", logy=True)
        

    def style_update(ax):
        ax.grid()
        ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
        ax.legend(loc="upper center")
        ax.set_ylabel("Relative Error (mean over trajectories, timesteps)")

    style_update(ax)
    plt.show()
plot_system_performance("spring")

# Individual Trajectory Visualization

In [ ]:
df = pandas\
.merge(aggregate_data, inferred_data, on="experiment_name")
df = pandas.merge(df, ground_truth_data, on=["experiment_name", "trajectory_number", "timestep_number"])


In [ ]:
run_df = df.loc[(df["system_name"] == "wave") &
                (df["method_name"] == "srnn") & 
                (df["network_depth"] == 2) & 
                (df["network_hidden_dim"] == 256) & 
                (df["num_train_trajectories"] == 500)]
run_df.head()

In [ ]:
single_run = run_df.loc[(run_df["trajectory_number"] == 0)][["inferred_data", "ground_truth_data"]]
inferred_data_ = np.stack([entry[0] for entry in single_run.values])
ground_truth_data_ = np.stack([entry[1] for entry in single_run.values])
def plot_t(t):
    plt.figure(figsize=(12, 8))
    plt.plot(inferred_data_[t, ...], "*")
    plt.plot(ground_truth_data_[t, ...], ":")
    plt.plot()

widgets.interact(plot_t, t=widgets.IntSlider(value=0, min=0, max=999))


# SRNN Plots

In [ ]:
fig, ax = plt.subplots(2, 2)
fig.set_figwidth(15)
fig.set_figheight(10)

for ax_, integrator_name in zip(itertools.chain(*ax), itertools.chain(["euler", "leapfrog"], ["rk4", "scipy-RK45"])):
    df = pandas\
            .merge(aggregate_data, inferred_data, on="experiment_name")\
            .query("system_name == 'spring' and method_name == 'srnn' and timestep_number == 1000 and integrator_name == '{}'".format(integrator_name))\
            .groupby(["num_train_trajectories", "integrator_timestep_size"])\
            .agg({"relerr_l2" : "mean"}).add_prefix("mean_").reset_index()
    df = df.pivot(index="integrator_timestep_size", columns=["num_train_trajectories"], values="mean_relerr_l2")

    for column in df.columns:
        ax_.plot(df[column], label=column)

    df = pandas\
            .merge(aggregate_data, inferred_data, on="experiment_name")\
            .query("system_name == 'spring' and method_name == 'integrator-baseline' and timestep_number == 1000 and integrator_name == '{}'".format(integrator_name))\
            .groupby(["integrator_timestep_size"])\
            .agg({"relerr_l2" : "mean"}).add_prefix("mean_").reset_index()

    ax_.plot(df["integrator_timestep_size"], df["mean_relerr_l2"], label="integrator-baseline")
    ax_.set_title(integrator_name)
    ax_.legend(loc="upper left")

In [ ]:
fig, ax = plt.subplots(2, 2)
fig.set_figwidth(15)
fig.set_figheight(10)

delta = aggregate_data["integrator_timestep_size"].unique()
timestep = ((1000 * delta.min())/delta).astype(np.int)
conversion = {dt : t for dt, t in zip(delta, timestep)}

for ax_, integrator_name in zip(itertools.chain(*ax), itertools.chain(["euler", "leapfrog"], ["rk4", "scipy-RK45"])):
    df = pandas.merge(aggregate_data, inferred_data, on="experiment_name")\
    .query("system_name == 'spring' and method_name == 'srnn' and integrator_name == '{}'".format(integrator_name))
    timesteps = df["integrator_timestep_size"].map(lambda x : conversion[x])
    df = df.loc[df["timestep_number"] == timesteps]
    inference_time_per_timestep = pandas.DataFrame({
        "experiment_name": aggregate_data["experiment_name"], 
        "inference_time_per_timestep" : (aggregate_data["inference_time"] / inferred_data["timestep_number"].max())
    })
    df = pandas.merge(df, inference_time_per_timestep, on="experiment_name")
    df["cpu_time"] = df.inference_time_per_timestep * df.timestep_number

    df = df.groupby(["num_train_trajectories", "cpu_time"])\
    .agg({"relerr_l2" : "mean"}).add_prefix("mean_").reset_index()
    group = df.groupby("num_train_trajectories")
    group_keys = list(group.groups.keys())
    for i, grp in enumerate(group.groups):
        mini_df = group.get_group(grp)
        ax_.plot(mini_df["cpu_time"], mini_df["mean_relerr_l2"], label=group_keys[i])

    df = pandas.merge(aggregate_data, inferred_data, on="experiment_name")\
    .query("system_name == 'spring' and method_name == 'integrator-baseline' and integrator_name == '{}'".format(integrator_name))
    timesteps = df["integrator_timestep_size"].map(lambda x : conversion[x])
    df = df.loc[df["timestep_number"] == timesteps]
    inference_time_per_timestep = pandas.DataFrame({
        "experiment_name": aggregate_data["experiment_name"], 
        "inference_time_per_timestep" : (aggregate_data["inference_time"] / inferred_data["timestep_number"].max())
    })
    df = pandas.merge(df, inference_time_per_timestep, on="experiment_name")
    df["cpu_time"] = df.inference_time_per_timestep * df.timestep_number

    df = df.groupby(["cpu_time"])\
    .agg({"relerr_l2" : "mean"}).add_prefix("mean_").reset_index()
    
    ax_.plot(df["cpu_time"], df["mean_relerr_l2"], label="integrator-baseline")
#     ax_.set(ylim=(0, 0.4))
    ax_.set_title(integrator_name)
    ax_.legend(loc="upper right")

# Training/Validation Loss Plots

In [ ]:
losses = {}
dirname = os.path.join(data_dir, "run/train/")
for root, folders, files in os.walk(dirname):
    if "train_stats.json" in files:
        with open(os.path.join(root, "train_stats.json"), "r") as file_:
            data = json.load(file_)
        train_loss = np.array([data["epoch_stats"][i]["train_total_loss"] / data["epoch_stats"][i]["train_loss_denom"] for i in range(400)])
        val_loss = np.array([data["epoch_stats"][i]["val_total_loss"] / data["epoch_stats"][i]["val_loss_denom"] for i in range(400)])
        losses[os.path.basename(root)[len("wave-architecture-test")+1:]] = [train_loss, val_loss]

In [ ]:
def plot_loss(key):
    plt.semilogy(losses[key][0])
    plt.semilogy(losses[key][1])

key = widgets.Dropdown(
    options=losses.keys(),
    description='Key:',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)

widgets.interact(plot_loss, key=key)

In [ ]:
ax = None
i = 0
plt.figure(figsize=(30, 20))
for d in [2, 3, 4]:
    for h in [200, 1024, 2048, 4096]:
        i += 1
        try:
            ax = plt.subplot(3, 4, i, sharex=ax, sharey=ax)
            key = f"hnn-train-wave-n500-t2500-n0.0-d{d}-h{h}_00001"
            plot_loss(key)
            plt.title(f"{d}-{h}")
        except KeyError as e:
            pass

In [ ]:
ax = None
i = 0
plt.figure(figsize=(30, 20))
for d in [2, 3, 4]:
    for h in [200, 1024, 2048, 4096]:
        i += 1
        try:
            ax = plt.subplot(3, 4, i, sharex=ax, sharey=ax)
            key = f"srnn-train-wave-n500-t2500-n0.0-d{d}-h{h}-ieuler_00001"
            plot_loss(key)
            plt.title(f"{d}-{h}")
        except KeyError as e:
            pass